In [27]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import ssl
from PIL import Image
import torchvision.transforms.functional as TF
import gradio as gr
import os
from PIL import Image

In [28]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.linear1 = nn.Linear(16 * 5 * 5, 120)
        self.linear2 = nn.Linear(120, 84)
        self.linear3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.maxpool(F.relu(self.conv1(x)))
        x = self.maxpool(F.relu(self.conv2(x)))

        x = x.reshape(-1, 16 * 5 * 5)

        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [29]:
class MyApp:
    def __init__(self):
        super().__init__()
        self.epoch = 50
        self.lr = 0.001
        self.model = MyModel()

        self.classes = ('squares', 'circles', 'triangles')
        self.labels = (0, 1, 2)

        self.transform = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
        ])

        self.dataset = torchvision.datasets.ImageFolder(root='shapes', transform=self.transform)
        self.dataloader = torch.utils.data.DataLoader(self.dataset, batch_size=64, shuffle=True, num_workers=2)

app = MyApp()

In [30]:
def train_model(device):
    loss_fonct = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(app.model.parameters(), lr=0.01)

    train_accuracies = np.zeros(app.epoch)
    train_loss = []

    for epoch in tqdm(range(app.epoch)):
        total_train, correct_train = 0, 0
        for batch_idx, batch in enumerate(tqdm(app.dataloader)):
            images, labels = batch
            images = images.to(device=device)
            labels = labels.to(device=device)

            output = app.model.forward(images)
            loss = loss_fonct(output, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(output.data, 1)
            total_train += labels.size(0)
            correct_train += predicted.eq(labels).sum().item()

            if batch_idx % 10 == 0:
                train_loss.append(loss.item())

            print("File Name:", app.dataset.samples[batch_idx][0])

        train_accuracies[epoch] = correct_train / total_train * 100
        print(correct_train / total_train * 100, "%\n")

    torch.save(app.model.state_dict(), "shapes-model.pth")

In [31]:
def predict(image):
    out = app.model(image.reshape(1, 3, 32, 32))
    _, pred = torch.max(out, dim=1)
    return app.classes[pred.item()]

In [32]:
def np_array_to_tensor_image(img, width=32, height=32, device='cpu'):
    image = Image.fromarray(img).convert('RGB').resize((width, height))
    image = transforms.Compose([
        transforms.ToTensor(),
    ])(image).unsqueeze(0)
    return image.to(device, torch.float)

In [33]:
def sketch_recognition(img):
    img = np_array_to_tensor_image(img)
    app.model.eval()
    with torch.no_grad():
        result = predict(img)
    app.model.train()
    return result

In [34]:
ssl._create_default_https_context = ssl._create_unverified_context

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

app.model.to(device)

MyModel(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (linear1): Linear(in_features=400, out_features=120, bias=True)
  (linear2): Linear(in_features=120, out_features=84, bias=True)
  (linear3): Linear(in_features=84, out_features=3, bias=True)
)

In [35]:
# if os.path.isfile("shapes-model.pth"):
#     app.model.load_state_dict(torch.load("shapes-model.pth"))
# else:
    # train_model(device)
train_model(device)
gr.Interface(fn=sketch_recognition, inputs=["sketchpad"], outputs="label").launch(share=True)


  2%|▏         | 1/50 [00:00<00:08,  5.49it/s]

File Name: shapes/circles/drawing(1).png
File Name: shapes/circles/drawing(10).png
File Name: shapes/circles/drawing(100).png
File Name: shapes/circles/drawing(11).png
File Name: shapes/circles/drawing(12).png
33.0 %



File Name: shapes/circles/drawing(1).png
File Name: shapes/circles/drawing(10).png
File Name: shapes/circles/drawing(100).png
File Name: shapes/circles/drawing(11).png
File Name: shapes/circles/drawing(12).png


  4%|▍         | 2/50 [00:00<00:07,  6.13it/s]

33.0 %



...

 98%|█████████▊| 49/50 [00:07<00:00,  6.39it/s]

33.33333333333333 %



File Name: shapes/circles/drawing(1).png
File Name: shapes/circles/drawing(10).png
File Name: shapes/circles/drawing(100).png


100%|██████████| 50/50 [00:07<00:00,  6.62it/s]


File Name: shapes/circles/drawing(11).png
File Name: shapes/circles/drawing(12).png
30.333333333333336 %

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://a3fe8df6-37ba-47ac.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
